<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 521 Files and Databases - Fall 2019</h1>
<h1 style="font-size: 250%;">Problem Set #3</h1>
<h1 style="font-size: 200%;">Version 1.1</h1>
<h1>Issued Tuesday 9/22/2020; Due Thursday 10/08/2020</h1>
</div>

**Homework Information:** The answer cells in this notebook contain metadata that enable it to be automatically parsed. Thus, you must enter your answers *only* into the cells provided. Do not delete any answer cell and/or add a new one to replace it. If you do so by accident, download a new copy of the problem set's notebook and transfer your answers into it. You can introduce new 'coding' cells in an effort to test approaches to problems, however, these introduced cells will not be considered when grading.

Finally, some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be accepted, but your lowest problem-set grade will be dropped. Feel free to get help from others, but the work you submit in should be your own.

In [ ]:
# Please fill out the entries in this Cell.
# By sigining it you are agreeing to the UNC honor code
class Student:
    first = "Wayne"
    last = "Ji"
    onyen = "rawreon"
    pid = "730168641"

In the following questions you will need to write Python3 programs to perform queries
    on and modify the <a href="http://csbio.unc.edu/mcmillan/Media/NCCOVID19.db" download>NCCOVID19.db</a>
    database used in Problem Set #2. **You are only allowed to import the standard "sqlite3"
    and the "time" Python packages.**

---
**Problem 1:** The file <a href="http://csbio.unc.edu/mcmillan/Media/NC2018Mortality.csv" download="NC2018Mortality.csv">NC2018Mortality.csv</a> contains a list of deaths by race and sex in each of North Carolina's counties. Write a Python3 program that creates a new table in the NCCOVID19.db database and then loads that table into the database using values from this file. The table should have the following schema:

<table style="border-collapse: collapse; border: 1px solid black;"><tbody>
<tr style="border-collapse: collapse; border: 1px solid black;">
    <th colspan="7" style="text-align:center; background-color: skyblue;">Mortality</th></tr>
<tr style="border-collapse: collapse; border: 1px solid black;">
    <td style="border-collapse: collapse; border: 1px solid black;"><u>fips</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;"><u>year</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;"><u>race</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;"><u>sex</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;">count</td>
</tr>
</tbody></table>

Do not parse the file using pandas or Python's built-in "csv" modules "reader" method. This will require that you open the file, read its contents, split them into lines, extract the header line,  

In [ ]:
import sqlite3
db = sqlite3.connect("NCCOVID19.db")
cursor = db.cursor()
cursor.execute("""
    CREATE TABLE Mortality (
        fips INTEGER,
        year INTEGER,
        race TEXT,
        sex TEXT,
        count INTEGER,
        PRIMARY KEY (fips, year, race, sex),
        FOREIGN KEY (fips) REFERENCES County(fips));    
""")
db.commit()

with open('NC2018Mortality.csv') as f:
    read_data = f.read()
test = read_data.split('\n')
test2 = [i.split(',') for i in test]
race_sex = test2[0][1:]
fips_name = {}
db = sqlite3.connect("NCCOVID19.db")
db.row_factory = sqlite3.Row
cursor = db.cursor()
cursor.execute("""
    SELECT fips, name
    FROM County
""")
for row in cursor.fetchall():
    fips_name[row['name'].upper()] = row['fips']
    

for row in test2[1:-1]:
    fips = fips_name[row[0].upper()]
    for x, row in zip(row[1:], race_sex):
        split = row.split('-')
        race = split[0]
        sex = split[1]
        cursor.execute("""INSERT INTO Mortality
                            VALUES(?, 2018,? ,? ,?)""", (fips, race, sex, int(x)))
        db.commit()
db.close()

---
**Problem 2:** Use the new table that you created in **Problem 1** to create the following Python3 function that estimates the number of 2018 deaths in each demographic category for a given county. These estimates are based on the assumption that *the number of mortality deaths in each demographic's age range is proportional to the 2018 population fraction in that same demogrphic's age range*. Your function should return a list of <code>(race,sex,agelo,agehi,count)</code> tuples, where the sum of all counts should match the total number of 2018 deaths in that county. All <code>counts</code> should be integers. All of your code must be within the body of the given function.

In [81]:
import sqlite3

def covidDemographicsEstimate(countyName):
    demographicGroup = []
    db = sqlite3.connect("NCCOVID19.db")
    db.row_factory = sqlite3.Row
    db.text_factory = str
    cursor = db.cursor()
    cursor.execute("""
    SELECT D.race, D.sex, D.agelo, D.agehi, D.count, M.count as deathsCount, T.totalDemographic, (D.count * 1.0 /T.totalDemographic) * M.count as percent
FROM Demographics D, Mortality M, County C,
(SELECT D.race, D.sex, sum(D.count) as totalDemographic
FROM Demographics D, County C
WHERE year = 2018 and D.fips = C.fips and C.name = ?
GROUP BY D.race, D.sex) as T
WHERE D.year = 2018 and D.sex = T.sex and D.race = T.race and D.fips = C.fips and D.fips = M.fips and D.race = M.race and D.sex = M.sex and C.name = ? 
GROUP BY agelo, D.race, D.sex
    """ ,(countyName, countyName))
    raceList = []
    sexList = []
    totalDeaths = []
    tempList = []
    for row in cursor:
        #if (row['race'] == 'white' and row['sex'] == 'female'): 
        demographicGroup.append((row['race'], row['sex'], row['agelo'], row['agehi'], int(row['percent'])))
        if row['race'] not in raceList:
            raceList.append(row['race'])
        if row['sex'] not in sexList:
            sexList.append(row['sex'])
        if row['deathsCount'] not in totalDeaths:
            totalDeaths.append(row['deathsCount'])
    
    
    return demographicGroup

In [82]:
covidDemographicsEstimate('Yancey')

[('white', 'female', 0, 2, 2),
 ('white', 'male', 0, 2, 3),
 ('white', 'female', 3, 4, 1),
 ('white', 'male', 3, 4, 2),
 ('white', 'female', 5, 5, 0),
 ('white', 'male', 5, 5, 1),
 ('white', 'female', 6, 9, 3),
 ('white', 'male', 6, 9, 5),
 ('white', 'female', 10, 13, 4),
 ('white', 'male', 10, 13, 6),
 ('white', 'female', 14, 14, 0),
 ('white', 'male', 14, 14, 1),
 ('white', 'female', 15, 15, 1),
 ('white', 'male', 15, 15, 1),
 ('white', 'female', 16, 17, 2),
 ('white', 'male', 16, 17, 3),
 ('white', 'female', 18, 19, 2),
 ('white', 'male', 18, 19, 2),
 ('white', 'female', 20, 24, 5),
 ('white', 'male', 20, 24, 6),
 ('white', 'female', 25, 34, 12),
 ('white', 'male', 25, 34, 17),
 ('white', 'female', 35, 44, 12),
 ('white', 'male', 35, 44, 14),
 ('white', 'female', 45, 54, 13),
 ('white', 'male', 45, 54, 18),
 ('white', 'female', 55, 59, 7),
 ('white', 'male', 55, 59, 9),
 ('white', 'female', 60, 64, 8),
 ('white', 'male', 60, 64, 9),
 ('white', 'female', 65, 74, 16),
 ('white', 'male

In [56]:
import iSQL

query = iSQL.parser("NCCOVID19.db")

SELECT D.race, D.sex, D.agelo, D.agehi, D.count, M.count as deathsCount, T.totalDemographic, (D.count * 1.0 /T.totalDemographic) * M.count as percent
FROM Demographics D, Mortality M, County C,
(SELECT D.race, D.sex, sum(D.count) as totalDemographic
FROM Demographics D, County C
WHERE year = 2018 and D.fips = C.fips and C.name = 'Yancey'
GROUP BY D.race, D.sex) as T
WHERE D.year = 2018 and D.sex = T.sex and D.race = T.race and D.fips = C.fips and D.fips = M.fips and D.race = M.race and D.sex = M.sex and C.name = 'Yancey'
GROUP BY agelo, D.race, D.sex

SyntaxError: invalid syntax (<ipython-input-56-f41cb0631575>, line 5)

---
**Problem 3:** In the space below write a SQL query that generates a list of (countyName, covidDeaths, totalDeaths2018, percentage) for all counties for which the 2020 Covid-19 deaths exceed 5% of the total deaths for that same county in 2018.

SELECT C.name, covidDeaths, totalDeaths2018, percentage
FROM County C,
(SELECT T.fips, covid_deaths as covidDeaths, deaths_2018 as totalDeaths2018, covid_deaths * 100.0 / deaths_2018 as percentage
FROM
(SELECT fips, sum(count) as deaths_2018
FROM Mortality
GROUP BY fips) as T,

(Select fips, sum(deaths) as covid_deaths
FROM Covid19
Group By fips) as P
WHERE T.fips = P.fips) as B
WHERE C.fips = B.fips and percentage > 5.0

In [41]:
import iSQL

query = iSQL.parser("NCCOVID19.db")


---
**Problem 4:** In the space below wirte a Python3 function that returns a list of all dates for which the given "fips" number accounts for at least the given "percent" of the total covid cases on that date. As an example, the NCCOVID19.db database includes Covid19 records that are not associated with any county. These records have a fips of 0. When your function is called with <code>AtLeastPercent(0, 10.0)</code> it should return the following list of dates:

     ['2020-06-05', '2020-06-10', '2020-06-15', '2020-08-12']

In [43]:
import sqlite3

def AtLeastPercent(fips, percent):
    dateList = []
    db = sqlite3.connect("NCCOVID19.db")
    db.row_factory = sqlite3.Row
    db.text_factory = str
    cursor = db.cursor()
    cursor.execute("""
    SELECT date
        FROM 
        (SELECT C.fips, C.date, C.cases, totalCases, C.cases *100.0/ totalCases as percentage
        FROM Covid19 C,
        (SELECT date, sum(cases) as totalCases
        FROM Covid19
        GROUP BY date) as T
        WHERE C.date = T.date
        GROUP BY C.fips, T.date) as P
        WHERE fips  = ? and percentage > ?""", (fips, percent))
    for x in cursor.fetchall():
        dateList.append(x['date'])
    return dateList

In [45]:
AtLeastPercent(37001, 2.0)

['2020-03-21',
 '2020-03-23',
 '2020-04-02',
 '2020-04-23',
 '2020-04-24',
 '2020-04-26',
 '2020-04-27',
 '2020-05-02',
 '2020-05-07',
 '2020-05-10',
 '2020-05-11',
 '2020-06-01',
 '2020-06-05',
 '2020-06-08',
 '2020-06-09',
 '2020-06-10',
 '2020-06-11',
 '2020-06-12',
 '2020-06-13',
 '2020-06-14',
 '2020-06-15',
 '2020-06-16',
 '2020-06-18',
 '2020-06-19',
 '2020-06-20',
 '2020-06-23',
 '2020-07-04',
 '2020-07-05',
 '2020-07-15',
 '2020-07-20',
 '2020-07-21',
 '2020-07-22',
 '2020-07-24',
 '2020-07-26',
 '2020-07-28',
 '2020-07-29',
 '2020-07-31',
 '2020-08-01',
 '2020-08-03',
 '2020-08-06',
 '2020-08-09',
 '2020-08-10',
 '2020-08-11',
 '2020-08-13',
 '2020-08-16',
 '2020-08-17',
 '2020-08-18',
 '2020-08-21',
 '2020-08-22',
 '2020-08-23',
 '2020-08-24',
 '2020-08-27']

---
**Problem 5:** The following query was designed to identify dates where two Counties reported the same number of cases on the same date:<br>
        <pre>
        SELECT C1.name, C2.name, V1.date, V1.cases
        FROM County C1, Covid19 V1, County C2, Covid19 V2
        WHERE C1.fips=V1.fips AND C2.fips=V2.fips
          AND V1.cases=V2.cases AND V1.date=V2.date AND V1.fips < V2.fips AND V1.cases >= 1
        </pre>
        
This query is slow using the default NCCOVID19.db. Write a Python3 program to execute and time
this query along with time to save all rows that it returns into a list for subsequent processing.
You should verify that your list actually extracts data from the rows. In the cell provided
below give the number of rows returned and the time required (no code please).

In [47]:
import time
import sqlite3
Q = """ SELECT C1.name, C2.name, V1.date, V1.cases
        FROM County C1, Covid19 V1, County C2, Covid19 V2
        WHERE C1.fips=V1.fips AND C2.fips=V2.fips
          AND V1.cases=V2.cases AND V1.date=V2.date AND V1.fips < V2.fips AND V1.cases >= 1"""
db = sqlite3.connect("NCCOVID19.db")
db.row_factory = sqlite3.Row
cursor = db.cursor()
start = time.time()
cursor.execute(Q)
twoCountyList = []
for row in cursor:
 twoCountyList.append((row['name'],row['name'],row['date'],row['cases']))
print("Number of rows (time) = %d (%6.4f secs)" % (len(twoCountyList),
time.time()-start))
for row in sorted(twoCountyList):
 print(row)

Number of rows (time) = 26826 (0.1800 secs)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-0

('Avery', 'Avery', '2020-05-19', 1)
('Avery', 'Avery', '2020-05-19', 1)
('Avery', 'Avery', '2020-05-19', 1)
('Avery', 'Avery', '2020-05-19', 1)
('Avery', 'Avery', '2020-05-22', 1)
('Avery', 'Avery', '2020-05-22', 1)
('Avery', 'Avery', '2020-05-22', 1)
('Avery', 'Avery', '2020-05-22', 1)
('Avery', 'Avery', '2020-05-22', 1)
('Avery', 'Avery', '2020-05-22', 1)
('Avery', 'Avery', '2020-06-05', 1)
('Avery', 'Avery', '2020-06-05', 1)
('Avery', 'Avery', '2020-06-05', 1)
('Avery', 'Avery', '2020-06-05', 1)
('Avery', 'Avery', '2020-06-05', 1)
('Avery', 'Avery', '2020-06-05', 1)
('Avery', 'Avery', '2020-06-05', 1)
('Avery', 'Avery', '2020-06-05', 1)
('Avery', 'Avery', '2020-06-08', 1)
('Avery', 'Avery', '2020-06-08', 1)
('Avery', 'Avery', '2020-06-08', 1)
('Avery', 'Avery', '2020-06-08', 1)
('Avery', 'Avery', '2020-06-08', 1)
('Avery', 'Avery', '2020-06-08', 1)
('Avery', 'Avery', '2020-06-08', 1)
('Avery', 'Avery', '2020-06-08', 1)
('Avery', 'Avery', '2020-06-08', 1)
('Avery', 'Avery', '2020-06-

('Bladen', 'Bladen', '2020-05-20', 3)
('Bladen', 'Bladen', '2020-05-20', 3)
('Bladen', 'Bladen', '2020-05-20', 3)
('Bladen', 'Bladen', '2020-05-20', 3)
('Bladen', 'Bladen', '2020-05-20', 3)
('Bladen', 'Bladen', '2020-05-20', 3)
('Bladen', 'Bladen', '2020-05-20', 3)
('Bladen', 'Bladen', '2020-05-20', 3)
('Bladen', 'Bladen', '2020-05-20', 3)
('Bladen', 'Bladen', '2020-05-20', 3)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-21', 2)
('Bladen', 'Bladen', '2020-05-22', 8)
('Bladen', 'Bladen', '2020-05-22', 8)
('Bladen', 'Bladen', '2020-05-28', 1)
('Bladen', 'Bladen', '2020-05-28', 1)
('Bladen', 'Bladen', '2020-05-28', 1)
('Bladen', '

('Caldwell', 'Caldwell', '2020-07-07', 12)
('Caldwell', 'Caldwell', '2020-07-08', 12)
('Caldwell', 'Caldwell', '2020-07-08', 12)
('Caldwell', 'Caldwell', '2020-07-09', 17)
('Caldwell', 'Caldwell', '2020-07-09', 17)
('Caldwell', 'Caldwell', '2020-07-09', 17)
('Caldwell', 'Caldwell', '2020-07-09', 17)
('Caldwell', 'Caldwell', '2020-07-10', 17)
('Caldwell', 'Caldwell', '2020-07-11', 21)
('Caldwell', 'Caldwell', '2020-07-14', 10)
('Caldwell', 'Caldwell', '2020-07-14', 10)
('Caldwell', 'Caldwell', '2020-07-14', 10)
('Caldwell', 'Caldwell', '2020-07-15', 22)
('Caldwell', 'Caldwell', '2020-07-15', 22)
('Caldwell', 'Caldwell', '2020-07-16', 22)
('Caldwell', 'Caldwell', '2020-07-18', 22)
('Caldwell', 'Caldwell', '2020-07-18', 22)
('Caldwell', 'Caldwell', '2020-07-19', 17)
('Caldwell', 'Caldwell', '2020-07-19', 17)
('Caldwell', 'Caldwell', '2020-07-19', 17)
('Caldwell', 'Caldwell', '2020-07-20', 18)
('Caldwell', 'Caldwell', '2020-07-21', 20)
('Caldwell', 'Caldwell', '2020-07-22', 26)
('Caldwell'

('Chowan', 'Chowan', '2020-06-11', 1)
('Chowan', 'Chowan', '2020-06-11', 1)
('Chowan', 'Chowan', '2020-06-27', 3)
('Chowan', 'Chowan', '2020-06-27', 3)
('Chowan', 'Chowan', '2020-06-27', 3)
('Chowan', 'Chowan', '2020-06-27', 3)
('Chowan', 'Chowan', '2020-06-27', 3)
('Chowan', 'Chowan', '2020-06-27', 3)
('Chowan', 'Chowan', '2020-06-28', 2)
('Chowan', 'Chowan', '2020-06-28', 2)
('Chowan', 'Chowan', '2020-06-28', 2)
('Chowan', 'Chowan', '2020-07-02', 1)
('Chowan', 'Chowan', '2020-07-02', 1)
('Chowan', 'Chowan', '2020-07-02', 1)
('Chowan', 'Chowan', '2020-07-02', 1)
('Chowan', 'Chowan', '2020-07-02', 1)
('Chowan', 'Chowan', '2020-07-02', 1)
('Chowan', 'Chowan', '2020-07-02', 1)
('Chowan', 'Chowan', '2020-07-03', 1)
('Chowan', 'Chowan', '2020-07-03', 1)
('Chowan', 'Chowan', '2020-07-03', 1)
('Chowan', 'Chowan', '2020-07-03', 1)
('Chowan', 'Chowan', '2020-07-03', 1)
('Chowan', 'Chowan', '2020-07-03', 1)
('Chowan', 'Chowan', '2020-07-03', 1)
('Chowan', 'Chowan', '2020-07-03', 1)
('Chowan', '

('Davidson', 'Davidson', '2020-05-06', 3)
('Davidson', 'Davidson', '2020-05-06', 3)
('Davidson', 'Davidson', '2020-05-06', 3)
('Davidson', 'Davidson', '2020-05-07', 5)
('Davidson', 'Davidson', '2020-05-08', 8)
('Davidson', 'Davidson', '2020-05-08', 8)
('Davidson', 'Davidson', '2020-05-08', 8)
('Davidson', 'Davidson', '2020-05-09', 2)
('Davidson', 'Davidson', '2020-05-09', 2)
('Davidson', 'Davidson', '2020-05-09', 2)
('Davidson', 'Davidson', '2020-05-09', 2)
('Davidson', 'Davidson', '2020-05-10', 1)
('Davidson', 'Davidson', '2020-05-10', 1)
('Davidson', 'Davidson', '2020-05-10', 1)
('Davidson', 'Davidson', '2020-05-10', 1)
('Davidson', 'Davidson', '2020-05-10', 1)
('Davidson', 'Davidson', '2020-05-10', 1)
('Davidson', 'Davidson', '2020-05-10', 1)
('Davidson', 'Davidson', '2020-05-10', 1)
('Davidson', 'Davidson', '2020-05-11', 3)
('Davidson', 'Davidson', '2020-05-11', 3)
('Davidson', 'Davidson', '2020-05-11', 3)
('Davidson', 'Davidson', '2020-05-11', 3)
('Davidson', 'Davidson', '2020-05-

('Gaston', 'Gaston', '2020-05-28', 1)
('Gaston', 'Gaston', '2020-05-28', 1)
('Gaston', 'Gaston', '2020-05-28', 1)
('Gaston', 'Gaston', '2020-05-28', 1)
('Gaston', 'Gaston', '2020-05-28', 1)
('Gaston', 'Gaston', '2020-05-29', 10)
('Gaston', 'Gaston', '2020-05-29', 10)
('Gaston', 'Gaston', '2020-05-29', 10)
('Gaston', 'Gaston', '2020-05-29', 10)
('Gaston', 'Gaston', '2020-05-30', 15)
('Gaston', 'Gaston', '2020-05-30', 15)
('Gaston', 'Gaston', '2020-06-01', 10)
('Gaston', 'Gaston', '2020-06-01', 10)
('Gaston', 'Gaston', '2020-06-02', 2)
('Gaston', 'Gaston', '2020-06-02', 2)
('Gaston', 'Gaston', '2020-06-02', 2)
('Gaston', 'Gaston', '2020-06-02', 2)
('Gaston', 'Gaston', '2020-06-02', 2)
('Gaston', 'Gaston', '2020-06-03', 9)
('Gaston', 'Gaston', '2020-06-03', 9)
('Gaston', 'Gaston', '2020-06-03', 9)
('Gaston', 'Gaston', '2020-06-04', 4)
('Gaston', 'Gaston', '2020-06-04', 4)
('Gaston', 'Gaston', '2020-06-04', 4)
('Gaston', 'Gaston', '2020-06-04', 4)
('Gaston', 'Gaston', '2020-06-04', 4)
('Ga

('Hertford', 'Hertford', '2020-05-20', 1)
('Hertford', 'Hertford', '2020-05-20', 1)
('Hertford', 'Hertford', '2020-05-20', 1)
('Hertford', 'Hertford', '2020-05-20', 1)
('Hertford', 'Hertford', '2020-05-20', 1)
('Hertford', 'Hertford', '2020-05-22', 1)
('Hertford', 'Hertford', '2020-05-22', 1)
('Hertford', 'Hertford', '2020-05-22', 1)
('Hertford', 'Hertford', '2020-05-23', 4)
('Hertford', 'Hertford', '2020-05-23', 4)
('Hertford', 'Hertford', '2020-05-23', 4)
('Hertford', 'Hertford', '2020-05-24', 1)
('Hertford', 'Hertford', '2020-05-24', 1)
('Hertford', 'Hertford', '2020-05-24', 1)
('Hertford', 'Hertford', '2020-05-24', 1)
('Hertford', 'Hertford', '2020-05-24', 1)
('Hertford', 'Hertford', '2020-05-24', 1)
('Hertford', 'Hertford', '2020-05-24', 1)
('Hertford', 'Hertford', '2020-05-24', 1)
('Hertford', 'Hertford', '2020-05-25', 3)
('Hertford', 'Hertford', '2020-05-25', 3)
('Hertford', 'Hertford', '2020-05-26', 3)
('Hertford', 'Hertford', '2020-05-26', 3)
('Hertford', 'Hertford', '2020-05-

('Macon', 'Macon', '2020-06-27', 3)
('Macon', 'Macon', '2020-06-27', 3)
('Macon', 'Macon', '2020-06-27', 3)
('Macon', 'Macon', '2020-06-29', 3)
('Macon', 'Macon', '2020-06-29', 3)
('Macon', 'Macon', '2020-06-30', 2)
('Macon', 'Macon', '2020-06-30', 2)
('Macon', 'Macon', '2020-06-30', 2)
('Macon', 'Macon', '2020-06-30', 2)
('Macon', 'Macon', '2020-07-05', 1)
('Macon', 'Macon', '2020-07-05', 1)
('Macon', 'Macon', '2020-07-05', 1)
('Macon', 'Macon', '2020-07-05', 1)
('Macon', 'Macon', '2020-07-05', 1)
('Macon', 'Macon', '2020-07-05', 1)
('Macon', 'Macon', '2020-07-06', 2)
('Macon', 'Macon', '2020-07-06', 2)
('Macon', 'Macon', '2020-07-07', 2)
('Macon', 'Macon', '2020-07-07', 2)
('Macon', 'Macon', '2020-07-07', 2)
('Macon', 'Macon', '2020-07-08', 5)
('Macon', 'Macon', '2020-07-09', 17)
('Macon', 'Macon', '2020-07-09', 17)
('Macon', 'Macon', '2020-07-10', 7)
('Macon', 'Macon', '2020-07-10', 7)
('Macon', 'Macon', '2020-07-12', 1)
('Macon', 'Macon', '2020-07-12', 1)
('Macon', 'Macon', '2020-0

('Pasquotank', 'Pasquotank', '2020-06-24', 4)
('Pasquotank', 'Pasquotank', '2020-06-24', 4)
('Pasquotank', 'Pasquotank', '2020-06-26', 8)
('Pasquotank', 'Pasquotank', '2020-06-28', 5)
('Pasquotank', 'Pasquotank', '2020-07-03', 5)
('Pasquotank', 'Pasquotank', '2020-07-04', 1)
('Pasquotank', 'Pasquotank', '2020-07-04', 1)
('Pasquotank', 'Pasquotank', '2020-07-04', 1)
('Pasquotank', 'Pasquotank', '2020-07-04', 1)
('Pasquotank', 'Pasquotank', '2020-07-05', 2)
('Pasquotank', 'Pasquotank', '2020-07-06', 1)
('Pasquotank', 'Pasquotank', '2020-07-06', 1)
('Pasquotank', 'Pasquotank', '2020-07-07', 4)
('Pasquotank', 'Pasquotank', '2020-07-07', 4)
('Pasquotank', 'Pasquotank', '2020-07-08', 8)
('Pasquotank', 'Pasquotank', '2020-07-08', 8)
('Pasquotank', 'Pasquotank', '2020-07-10', 1)
('Pasquotank', 'Pasquotank', '2020-07-10', 1)
('Pasquotank', 'Pasquotank', '2020-07-10', 1)
('Pasquotank', 'Pasquotank', '2020-07-10', 1)
('Pasquotank', 'Pasquotank', '2020-07-10', 1)
('Pasquotank', 'Pasquotank', '2020

---
**Problem 6:** Next create an index to improve the performance of this query. In the cell provided below give
the "CREATE INDEX" statement that you found provides the best performance improvement. Also include
the time required for the query after creating the index you chose.

In [53]:
import time
import sqlite3
Q = """ SELECT C1.name, C2.name, V1.date, V1.cases
        FROM County C1, Covid19 V1, County C2, Covid19 V2
        WHERE C1.fips=V1.fips AND C2.fips=V2.fips
          AND V1.cases=V2.cases AND V1.date=V2.date AND V1.fips < V2.fips AND V1.cases >= 1"""
db = sqlite3.connect("NCCOVID19.db")
db.row_factory = sqlite3.Row
cursor = db.cursor()
cursor.execute("""
    DROP INDEX IF EXISTS myIndex
""")
cursor.execute("""
    CREATE INDEX myIndex
    ON Covid19 (cases, date, fips)
""")
start = time.time()
cursor.execute(Q)
twoCountyList = []
for row in cursor:
 twoCountyList.append((row['name'],row['name'],row['date'],row['cases']))
print("Number of rows (time) = %d (%6.4f secs)" % (len(twoCountyList),
time.time()-start))
for row in sorted(twoCountyList):
 print(row)

Number of rows (time) = 26826 (0.1170 secs)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-21', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-23', 1)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-03-26', 2)
('Alamance', 'Alamance', '2020-0

('Ashe', 'Ashe', '2020-05-21', 3)
('Ashe', 'Ashe', '2020-05-21', 3)
('Ashe', 'Ashe', '2020-05-21', 3)
('Ashe', 'Ashe', '2020-05-23', 2)
('Ashe', 'Ashe', '2020-05-23', 2)
('Ashe', 'Ashe', '2020-05-23', 2)
('Ashe', 'Ashe', '2020-05-23', 2)
('Ashe', 'Ashe', '2020-05-23', 2)
('Ashe', 'Ashe', '2020-05-23', 2)
('Ashe', 'Ashe', '2020-05-23', 2)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-24', 1)
('Ashe', 'Ashe', '2020-05-29', 2)
('Ashe', 'Ashe', '2020-05-29', 2)
('Ashe', 'Ashe', '2020-05-29', 2)
('Ashe', 'Ashe', '2020-05-29', 2)
('Ashe', 'Ashe', '2020-05-29', 2)
('Ashe', 'Ashe

('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-17', 1)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-18', 2)
('Bladen', 'Bladen', '2020-05-19', 5)
('Bladen', 'Bladen', '2020-05-19', 5)
('Bladen', 'Bladen', '2020-05-19', 5)
('Bladen', 'Bladen', '2020-05-19', 5)
('Bladen', '

('Caldwell', 'Caldwell', '2020-06-20', 8)
('Caldwell', 'Caldwell', '2020-06-20', 8)
('Caldwell', 'Caldwell', '2020-06-21', 10)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-22', 1)
('Caldwell', 'Caldwell', '2020-06-23', 7)
('Caldwell', 'Caldwell', '2020-06-23', 7)
('Caldwell', 'Caldwell', '2020-06-24', 6)
('Caldwell', 'Caldwell', '2020-06-24', 6)
('Caldwell', 'Caldwell', '2020-06-25', 7)
('Caldwell', 'Caldwell', '2020-06-26', 13)
('Caldwell', 'Caldwell', '2020-06-26', 13)
('Caldwell', 'Caldwell', '2020-06-27', 10)
('Caldwell', 'Caldwell', '2020

('Cherokee', 'Cherokee', '2020-03-30', 1)
('Cherokee', 'Cherokee', '2020-03-30', 1)
('Cherokee', 'Cherokee', '2020-03-30', 1)
('Cherokee', 'Cherokee', '2020-03-30', 1)
('Cherokee', 'Cherokee', '2020-03-30', 1)
('Cherokee', 'Cherokee', '2020-03-30', 1)
('Cherokee', 'Cherokee', '2020-03-30', 1)
('Cherokee', 'Cherokee', '2020-03-30', 1)
('Cherokee', 'Cherokee', '2020-03-30', 1)
('Cherokee', 'Cherokee', '2020-04-01', 1)
('Cherokee', 'Cherokee', '2020-04-01', 1)
('Cherokee', 'Cherokee', '2020-04-01', 1)
('Cherokee', 'Cherokee', '2020-04-01', 1)
('Cherokee', 'Cherokee', '2020-04-01', 1)
('Cherokee', 'Cherokee', '2020-04-01', 1)
('Cherokee', 'Cherokee', '2020-04-01', 1)
('Cherokee', 'Cherokee', '2020-04-01', 1)
('Cherokee', 'Cherokee', '2020-04-01', 1)
('Cherokee', 'Cherokee', '2020-04-01', 1)
('Cherokee', 'Cherokee', '2020-04-07', 1)
('Cherokee', 'Cherokee', '2020-04-07', 1)
('Cherokee', 'Cherokee', '2020-04-07', 1)
('Cherokee', 'Cherokee', '2020-04-07', 1)
('Cherokee', 'Cherokee', '2020-04-

...

('Edgecombe', 'Edgecombe', '2020-06-20', 6)
('Edgecombe', 'Edgecombe', '2020-06-20', 6)
('Edgecombe', 'Edgecombe', '2020-06-20', 6)
('Edgecombe', 'Edgecombe', '2020-06-20', 6)
('Edgecombe', 'Edgecombe', '2020-06-20', 6)
('Edgecombe', 'Edgecombe', '2020-06-21', 5)
('Edgecombe', 'Edgecombe', '2020-06-21', 5)
('Edgecombe', 'Edgecombe', '2020-06-21', 5)
('Edgecombe', 'Edgecombe', '2020-06-21', 5)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-22', 1)
('Edgecombe', 'Edgecombe', '2020-06-23', 2)
('Edgecombe', 'Edgecombe', '2020-06-23', 2)
('Edgecombe', 'Edgecombe', '2020

('Halifax', 'Halifax', '2020-05-01', 1)
('Halifax', 'Halifax', '2020-05-01', 1)
('Halifax', 'Halifax', '2020-05-01', 1)
('Halifax', 'Halifax', '2020-05-01', 1)
('Halifax', 'Halifax', '2020-05-01', 1)
('Halifax', 'Halifax', '2020-05-01', 1)
('Halifax', 'Halifax', '2020-05-04', 2)
('Halifax', 'Halifax', '2020-05-04', 2)
('Halifax', 'Halifax', '2020-05-04', 2)
('Halifax', 'Halifax', '2020-05-04', 2)
('Halifax', 'Halifax', '2020-05-04', 2)
('Halifax', 'Halifax', '2020-05-05', 5)
('Halifax', 'Halifax', '2020-05-06', 4)
('Halifax', 'Halifax', '2020-05-06', 4)
('Halifax', 'Halifax', '2020-05-06', 4)
('Halifax', 'Halifax', '2020-05-06', 4)
('Halifax', 'Halifax', '2020-05-06', 4)
('Halifax', 'Halifax', '2020-05-07', 4)
('Halifax', 'Halifax', '2020-05-08', 5)
('Halifax', 'Halifax', '2020-05-09', 3)
('Halifax', 'Halifax', '2020-05-09', 3)
('Halifax', 'Halifax', '2020-05-09', 3)
('Halifax', 'Halifax', '2020-05-11', 3)
('Halifax', 'Halifax', '2020-05-11', 3)
('Halifax', 'Halifax', '2020-05-11', 3)


('Johnston', 'Johnston', '2020-08-10', 11)
('Johnston', 'Johnston', '2020-08-10', 11)
('Johnston', 'Johnston', '2020-08-15', 25)
('Johnston', 'Johnston', '2020-08-16', 11)
('Johnston', 'Johnston', '2020-08-17', 9)
('Johnston', 'Johnston', '2020-08-17', 9)
('Johnston', 'Johnston', '2020-08-22', 24)
('Johnston', 'Johnston', '2020-08-24', 23)
('Johnston', 'Johnston', '2020-08-26', 19)
('Johnston', 'Johnston', '2020-08-26', 19)
('Johnston', 'Johnston', '2020-08-27', 53)
('Jones', 'Jones', '2020-04-09', 3)
('Jones', 'Jones', '2020-04-09', 3)
('Jones', 'Jones', '2020-04-09', 3)
('Jones', 'Jones', '2020-04-09', 3)
('Jones', 'Jones', '2020-04-11', 3)
('Jones', 'Jones', '2020-04-11', 3)
('Jones', 'Jones', '2020-04-11', 3)
('Jones', 'Jones', '2020-04-12', 1)
('Jones', 'Jones', '2020-04-12', 1)
('Jones', 'Jones', '2020-04-12', 1)
('Jones', 'Jones', '2020-04-12', 1)
('Jones', 'Jones', '2020-04-12', 1)
('Jones', 'Jones', '2020-04-12', 1)
('Jones', 'Jones', '2020-04-12', 1)
('Jones', 'Jones', '2020-

('Montgomery', 'Montgomery', '2020-07-17', 10)
('Montgomery', 'Montgomery', '2020-07-17', 10)
('Montgomery', 'Montgomery', '2020-07-19', 7)
('Montgomery', 'Montgomery', '2020-07-20', 4)
('Montgomery', 'Montgomery', '2020-07-20', 4)
('Montgomery', 'Montgomery', '2020-07-21', 2)
('Montgomery', 'Montgomery', '2020-07-21', 2)
('Montgomery', 'Montgomery', '2020-07-21', 2)
('Montgomery', 'Montgomery', '2020-07-23', 3)
('Montgomery', 'Montgomery', '2020-07-23', 3)
('Montgomery', 'Montgomery', '2020-07-23', 3)
('Montgomery', 'Montgomery', '2020-07-27', 17)
('Montgomery', 'Montgomery', '2020-07-28', 21)
('Montgomery', 'Montgomery', '2020-07-31', 6)
('Montgomery', 'Montgomery', '2020-08-01', 16)
('Montgomery', 'Montgomery', '2020-08-02', 2)
('Montgomery', 'Montgomery', '2020-08-02', 2)
('Montgomery', 'Montgomery', '2020-08-02', 2)
('Montgomery', 'Montgomery', '2020-08-02', 2)
('Montgomery', 'Montgomery', '2020-08-02', 2)
('Montgomery', 'Montgomery', '2020-08-03', 7)
('Montgomery', 'Montgomery', 

('Rutherford', 'Rutherford', '2020-03-31', 3)
('Rutherford', 'Rutherford', '2020-04-02', 2)
('Rutherford', 'Rutherford', '2020-04-02', 2)
('Rutherford', 'Rutherford', '2020-04-02', 2)
('Rutherford', 'Rutherford', '2020-04-02', 2)
('Rutherford', 'Rutherford', '2020-04-03', 1)
('Rutherford', 'Rutherford', '2020-04-04', 2)
('Rutherford', 'Rutherford', '2020-04-04', 2)
('Rutherford', 'Rutherford', '2020-04-05', 2)
('Rutherford', 'Rutherford', '2020-04-09', 3)
('Rutherford', 'Rutherford', '2020-04-10', 3)
('Rutherford', 'Rutherford', '2020-04-10', 3)
('Rutherford', 'Rutherford', '2020-04-10', 3)
('Rutherford', 'Rutherford', '2020-04-13', 3)
('Rutherford', 'Rutherford', '2020-04-13', 3)
('Rutherford', 'Rutherford', '2020-04-15', 2)
('Rutherford', 'Rutherford', '2020-04-15', 2)
('Rutherford', 'Rutherford', '2020-04-15', 2)
('Rutherford', 'Rutherford', '2020-04-19', 10)
('Rutherford', 'Rutherford', '2020-04-28', 1)
('Rutherford', 'Rutherford', '2020-04-28', 1)
('Rutherford', 'Rutherford', '202

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set.

You can submit it as many times as you would like, but only the last submitted version will be graded. If the last submission was after the due date the final score will be reduced accordingly. 